<a href="https://colab.research.google.com/github/vharshitha19/ML-PROJECTS/blob/main/Fraud_Detection/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

data_folder = "data"   # this folder came from your zip

all_data = []

for file in os.listdir(data_folder):
    if file.endswith(".pkl"):
        df = pd.read_pickle(os.path.join(data_folder, file))
        all_data.append(df)

data = pd.concat(all_data, ignore_index=True)

print("Dataset shape:", data.shape)
print(data.head())


Dataset shape: (1754155, 9)
   TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  TX_AMOUNT  \
0          220747 2018-04-24 00:00:08        3853        8513      40.37   
1          220748 2018-04-24 00:01:00         723        1469      75.08   
2          220749 2018-04-24 00:01:20        3395        5545      30.04   
3          220750 2018-04-24 00:02:40         350        8902      92.20   
4          220751 2018-04-24 00:03:44        1348        6654      51.77   

  TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  
0         1987208           23         0                  0  
1         1987260           23         0                  0  
2         1987280           23         0                  0  
3         1987360           23         1                  3  
4         1987424           23         0                  0  


In [ ]:
# Convert datetime column
data['TX_DATETIME'] = pd.to_datetime(data['TX_DATETIME'])

# Create time features (helps fraud patterns)
data['hour'] = data['TX_DATETIME'].dt.hour
data['day'] = data['TX_DATETIME'].dt.day
data['month'] = data['TX_DATETIME'].dt.month

# Drop original datetime
data.drop('TX_DATETIME', axis=1, inplace=True)

print(data.head())


   TRANSACTION_ID CUSTOMER_ID TERMINAL_ID  TX_AMOUNT TX_TIME_SECONDS  \
0          220747        3853        8513      40.37         1987208   
1          220748         723        1469      75.08         1987260   
2          220749        3395        5545      30.04         1987280   
3          220750         350        8902      92.20         1987360   
4          220751        1348        6654      51.77         1987424   

  TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  hour  day  month  
0           23         0                  0     0   24      4  
1           23         0                  0     0   24      4  
2           23         0                  0     0   24      4  
3           23         1                  3     0   24      4  
4           23         0                  0     0   24      4  


In [ ]:
X = data.drop('TX_FRAUD', axis=1)
# Ensure TX_DATETIME is removed if it's still present
if 'TX_DATETIME' in X.columns:
    X = X.drop('TX_DATETIME', axis=1)
y = data['TX_FRAUD']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

print("Model trained successfully!")


Model trained successfully!


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    347895
           1       1.00      1.00      1.00      2936

    accuracy                           1.00    350831
   macro avg       1.00      1.00      1.00    350831
weighted avg       1.00      1.00      1.00    350831



In [ ]:
import pandas as pd

new_transaction = pd.DataFrame([{
    'TRANSACTION_ID': 123456,
    'CUSTOMER_ID': 1000, # Added missing feature with a placeholder value
    'TERMINAL_ID': 300,
    'TX_AMOUNT': 50,
    'TX_TIME_SECONDS': 100000, # Added missing feature with a placeholder value
    'TX_TIME_DAYS': 10, # Added missing feature with a placeholder value
    'TX_FRAUD_SCENARIO': 0, # Added missing feature with a placeholder value
    'hour': 14,
    'day': 10,
    'month': 4
}])

prediction = model.predict(new_transaction)

if prediction[0] == 1:
    print("⚠ Fraud Transaction")
else:
    print(" Legit Transaction")

 Legit Transaction
